In [0]:
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [95]:
import keras
from keras.datasets import cifar10
k = 12
L = 20
batch_size = 128
num_epochs = 50
num_classes = 10
from keras.layers import Input, Dense, BatchNormalization, Activation, AveragePooling2D, Conv2D, Input, Flatten, concatenate
from keras.models import Model
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# Convert class vectors to binary class matrices.
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [0]:
# Dense Net Block
def dense_net_model(inp):
    temp = inp
    for i in range(L):
        bnorm_out = BatchNormalization(axis=1)(temp)
        relu_out = Activation('relu')(bnorm_out)
        conv_out = Conv2D(k, (3, 3), padding='same')(relu_out)
        temp = concatenate([temp, conv_out], axis=-1)
    print(temp.shape)    
    return temp
        

In [0]:
#Transition Block
def transition_model(inp, channels):
  bnorm_out = BatchNormalization(axis=1)(inp)
  conv_out = Conv2D(channels, (1, 1), padding='same')(bnorm_out)
  avg_out = AveragePooling2D(pool_size=(2, 2))(conv_out)
  print(avg_out.shape)    
  return avg_out

In [0]:
def create_full_model(inputShape):
  input_img = Input(shape=inputShape)
  start = Conv2D(16, (7, 7), padding='same')(input_img)
  print(start.shape)
  dense_out = dense_net_model(start)
  trans_out = transition_model(dense_out, 56)
  dense_out = dense_net_model(trans_out)
  trans_out = transition_model(dense_out, 28)
  dense_out = dense_net_model(trans_out)
  bnorm_out = BatchNormalization(axis=1)(dense_out)
  relu_out = Activation('relu')(bnorm_out)
  avg_out = AveragePooling2D(pool_size=(3, 3))(relu_out)
  flatten = Flatten()(avg_out)
  out = Dense(10, activation = 'softmax', name='fc')(flatten)
  return Model(input_img, out)

In [96]:
from keras.optimizers import SGD
model = create_full_model(x_train[0].shape)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=num_epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

(?, 32, 32, 16)
(?, 32, 32, 256)
(?, 16, 16, 56)
(?, 16, 16, 296)
(?, 8, 8, 28)
(?, 8, 8, 268)
fs (?, ?)
os (?, 10)
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 534s 11ms/step - loss: 1.8258 - acc: 0.3350 - val_loss: 1.4841 - val_acc: 0.4736
Epoch 2/50
50000/50000 [==============================] - 494s 10ms/step - loss: 1.3654 - acc: 0.5077 - val_loss: 1.2462 - val_acc: 0.5551
Epoch 3/50
50000/50000 [==============================] - 494s 10ms/step - loss: 1.1779 - acc: 0.5793 - val_loss: 1.1430 - val_acc: 0.5910
Epoch 4/50
50000/50000 [==============================] - 493s 10ms/step - loss: 1.0357 - acc: 0.6339 - val_loss: 1.0200 - val_acc: 0.6352
Epoch 5/50
50000/50000 [==============================] - 494s 10ms/step - loss: 0.9213 - acc: 0.6761 - val_loss: 0.9187 - val_acc: 0.6755
Epoch 6/50
50000/50000 [==============================] - 493s 10ms/step - loss: 0.8230 - acc: 0.7115 - val_loss: 0.9515 - val_acc: 0.6697


KeyboardInterrupt: ignored

In [105]:
!ls

datalab  sample_data
